# AMBIENTE READR

In [ ]:
import pandas as pd
import numpy as np
import folium
from shapely.geometry import Polygon
from pyproj import Transformer
import matplotlib.colors as mcolors
import pyreadr
result = pyreadr.read_r('/Volumes/ssd_externo/UEL MESTRADO 2020/Dissertação/Simulações em R/GAMLSS/pred_geral')
df = result[None]

print(f"Total de pontos: {len(df)}")
print(df.head())

In [ ]:
# Calcular distâncias entre pontos consecutivos
import numpy as np

# Pegar primeiros pontos para análise
x_coords = df['x'].values[:100]
y_coords = df['y'].values[:100]

# Calcular diferenças entre pontos consecutivos
diff_x = np.diff(x_coords)
diff_y = np.diff(y_coords)

# Distâncias únicas
unique_diff_x = np.unique(diff_x[diff_x != 0])
unique_diff_y = np.unique(diff_y[diff_y != 0])

print("Diferenças únicas em X (metros):")
print(unique_diff_x[:10])

print("\nDiferenças únicas em Y (metros):")
print(unique_diff_y[:10])

# Calcular distância euclidiana entre pontos adjacentes
distances = []
for i in range(len(df)-1):
    dx = df['x'].iloc[i+1] - df['x'].iloc[i]
    dy = df['y'].iloc[i+1] - df['y'].iloc[i]
    if dx != 0 or dy != 0:
        dist = np.sqrt(dx**2 + dy**2)
        distances.append(dist)

distances = np.array(distances)
print(f"\nDistância mínima: {distances.min():.2f}m")
print(f"Distância mais comum: {np.median(distances[distances > 0]):.2f}m")

In [7]:
# 1. Paleta de cores
def tim_colors(n=1000):
    colors = [
        '#000080', '#0000CC', '#0040FF', '#0080FF', '#00BFFF', '#00FFFF',
        '#00FFBF', '#00FF80', '#00FF40', '#00FF00', '#40FF00', '#80FF00',
        '#BFFF00', '#FFFF00', '#FFBF00', '#FF8000', '#FF4000', '#FF0000'
    ]
    cmap = mcolors.LinearSegmentedColormap.from_list("tim", colors, N=n)
    return [mcolors.rgb2hex(cmap(i)) for i in np.linspace(0, 1, n)]

pal14 = tim_colors(1000)

# 2. Parâmetros do grid
grid_spacing = 50  # metros (50x50m)
half_spacing = 25  # metros

# 3. Lista de períodos temporais
periodos = [col for col in df.columns if col not in ['x', 'y']]
print(f"Total de períodos: {len(periodos)}")
print(f"Períodos: {periodos}")

# 4. Dicionário para mapear nomes de períodos para labels
def format_period_label(periodo):
    """Converte 'jan.2000' para 'January 2000'"""
    mes_map = {
        'jan': 'January', 'jul': 'July'
    }
    mes, ano = periodo.split('.')
    return f"{mes_map[mes]} {ano}"

# 5. Criar quadrados em UTM (uma vez, reutilizar)
def create_square_utm(utm_x, utm_y, half_size):
    return Polygon([
        (utm_x - half_size, utm_y - half_size),
        (utm_x + half_size, utm_y - half_size),
        (utm_x + half_size, utm_y + half_size),
        (utm_x - half_size, utm_y + half_size),
        (utm_x - half_size, utm_y - half_size)
    ])

print("Criando quadrados em UTM (uma vez)...")
utm_polygons = []
for idx, row in df.iterrows():
    square_utm = create_square_utm(row['x'], row['y'], half_spacing)
    utm_polygons.append(square_utm)

# 6. Converter para lat/lon (uma vez, reutilizar)
print("Convertendo para lat/lon (uma vez)...")
transformer = Transformer.from_crs("EPSG:29192", "EPSG:4326", always_xy=True)

def transform_polygon_to_latlon(polygon_utm):
    coords_utm = list(polygon_utm.exterior.coords)
    coords_latlon = []
    for x_utm, y_utm in coords_utm:
        lon, lat = transformer.transform(x_utm, y_utm)
        coords_latlon.append([lat, lon])
    return coords_latlon

latlon_polygons = []
for polygon_utm in utm_polygons:
    latlon_polygon = transform_polygon_to_latlon(polygon_utm)
    latlon_polygons.append(latlon_polygon)

print(f"Geometrias preparadas: {len(latlon_polygons)} polígonos")

# Centro do mapa (calcular uma vez)
center_utm_x = df['x'].mean()
center_utm_y = df['y'].mean()
center_lon, center_lat = transformer.transform(center_utm_x, center_utm_y)

# 7. LOOP PRINCIPAL: Gerar HTML para cada período
output_dir = '/Users/fjcosta/Documents/landCoverlandValue/landvalue/'
os.makedirs(output_dir, exist_ok=True)

for periodo in periodos:
    print(f"\n{'='*60}")
    print(f"Processando período: {periodo}")
    print(f"{'='*60}")
    
    periodo_label = format_period_label(periodo)
    
    # Obter min e max dos valores
    min_value = df[periodo].min()
    max_value = df[periodo].max()
    value_range = max_value - min_value
    
    print(f"Valores - Min: {min_value:.2f}, Max: {max_value:.2f}")
    
    # Criar breaks
    breaks = np.linspace(min_value, max_value, 18)
    
    def get_color_from_value(value):
        normalized = (value - min_value) / value_range
        color_index = int(normalized * (len(pal14) - 1))
        color_index = max(0, min(color_index, len(pal14) - 1))
        return pal14[color_index]
    
    # Criar mapa
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr='Google Satellite'
    )
    
    # Criar GeoJSON
    print("Criando GeoJSON...")
    features = []
    for idx in range(len(latlon_polygons)):
        polygon_coords = latlon_polygons[idx]
        value = df.iloc[idx][periodo]
        color = get_color_from_value(value)
        
        geojson_coords = [[[coord[1], coord[0]] for coord in polygon_coords]]
        
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": geojson_coords
            },
            "properties": {
                "fillColor": color,
                "color": color,
                "weight": 0,
                "fillOpacity": 0.7,
                "popup": f"Median unit value: R$ {value:.2f}/m²"
            }
        }
        features.append(feature)
    
    geojson_data = {
        "type": "FeatureCollection",
        "features": features
    }
    
    # Adicionar ao mapa
    folium.GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': feature['properties']['fillColor'],
            'color': feature['properties']['color'],
            'weight': feature['properties']['weight'],
            'fillOpacity': feature['properties']['fillOpacity'],
            'opacity': 0
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['popup'],
            aliases=[''],
            labels=False
        )
    ).add_to(m)
    
    # Criar legenda
    legend_html = f"""
    <div style="position: fixed; 
               top: 10px; right: 10px; width: 280px; height: 570px; 
               background-color: white; border:2px solid grey; z-index:9999; 
               font-size:12px; padding: 10px; overflow-y: scroll;">
    <p><b>Londrina Urban Land Median Values for {periodo_label}</b></p>
    <table style="font-size:10px; border-collapse: collapse;">
    """
    
    for i in range(len(breaks)-1):
        color_index = int((i / (len(breaks)-2)) * (len(pal14)-1))
        color = pal14[color_index]
        value_min = breaks[i]
        value_max = breaks[i+1]
        
        legend_html += f"""
        <tr>
            <td style="background-color:{color}; width:20px; height:10px; border:1px solid #ccc;"></td>
            <td style="padding-left:8px; vertical-align:middle;">R$ {value_min:.2f}/m² - R$ {value_max:.2f}/m²</td>
        </tr>
        """
    
    legend_html += f"""
    </table>
    <div style="font-size:10px; margin-top:15px; color:#666; line-height:1.4;">
    <p style="margin:0; font-weight:bold;">Specifications:</p>
    <ul style="margin:5px 0; padding-left:15px;">
        <li>Spatial Resolution: {grid_spacing}m × {grid_spacing}m</li>
        <li>Coordinate System: EPSG:29192 (SAD69 UTM 22S)</li>
    </ul>
    </div>
    </div>
    """
    
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Salvar mapa
    output_filename = periodo.replace('.', '_') + '.html'
    output_path = os.path.join(output_dir, output_filename)
    m.save(output_path)
    
    # Verificar tamanho
    file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"✅ Salvo: {output_filename}")
    print(f"📊 Tamanho: {file_size_mb:.2f} MB")
    
    if file_size_mb > 60:
        print(f"⚠️ ATENÇÃO: Arquivo maior que 60 MB!")

print(f"\n{'='*60}")
print("✅ TODOS OS PERÍODOS PROCESSADOS!")
print(f"{'='*60}")
print(f"Total de arquivos gerados: {len(periodos)}")
print(f"Diretório: {output_dir}")

Total de períodos: 44
Períodos: ['jan.2000', 'jul.2000', 'jan.2001', 'jul.2001', 'jan.2002', 'jul.2002', 'jan.2003', 'jul.2003', 'jan.2004', 'jul.2004', 'jan.2005', 'jul.2005', 'jan.2006', 'jul.2006', 'jan.2007', 'jul.2007', 'jan.2008', 'jul.2008', 'jan.2009', 'jul.2009', 'jan.2010', 'jul.2010', 'jan.2011', 'jul.2011', 'jan.2012', 'jul.2012', 'jan.2013', 'jul.2013', 'jan.2014', 'jul.2014', 'jan.2015', 'jul.2015', 'jan.2016', 'jul.2016', 'jan.2017', 'jul.2017', 'jan.2018', 'jul.2018', 'jan.2019', 'jul.2019', 'jan.2020', 'jul.2020', 'jan.2021', 'jul.2021']
Criando quadrados em UTM (uma vez)...
Convertendo para lat/lon (uma vez)...
Geometrias preparadas: 89089 polígonos

Processando período: jan.2000
Valores - Min: 6.84, Max: 157.26
Criando GeoJSON...
✅ Salvo: jan_2000.html
📊 Tamanho: 38.95 MB

Processando período: jul.2000
Valores - Min: 7.45, Max: 167.62
Criando GeoJSON...
✅ Salvo: jul_2000.html
📊 Tamanho: 38.95 MB

Processando período: jan.2001
Valores - Min: 8.09, Max: 178.61
Criando 

In [ ]:
import pandas as pd

# Supondo que df é seu DataFrame
coluna_jan2000 = df['jan.2000']

# Ou acessar várias colunas
coordenadas = df[['x', 'y']]

# Ver valores únicos
print(df['jan.2000'].unique())

# Estatísticas
print(df['jan.2000'].describe())

# Primeira linha de uma coluna
print(df['jan.2000'].iloc[0])

# Converter para lista
lista_valores = df['jan.2000'].tolist()

# Filtrar linhas baseado em coluna
df_filtrado = df[df['jan.2000'] > 90]